# Classification des données en utilisant le SVM

## Importation des données

In [ ]:
import pymongo
import sklearn as sk
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from itertools import combinations

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
user_collection = db["users_labeled"]

In [ ]:
users = list(user_collection.find({}))
users = pd.DataFrame(users)

In [ ]:
attributs_to_drop = ['suspicious_score', 'suspicious_fields', 'user_id', "friends_count","followers_count","tweet_frequency"]
users = users.drop(columns=attributs_to_drop)
print(users.columns)

In [ ]:
Y=users.label
X=users.drop(columns=["label", "_id"])
attributs=[att for att in X.columns]

## Quelques statistiques sur nos utilisateurs précédemment labellisés
En examinant la distribution de chaque attribut nous sommes en mesure d'identifier les tendances et les schémas qui se dégagent.Cette démarche vise à obtenir une compréhension approfondie/préliminaire des caractéristiques de nos utilisateurs atypique et non.

Nous allons dans un premier temps récupérer les données non standardisé de nos utilisateur labélisé.

In [ ]:
pipeline = [
  {
    "$project": {
      "label": 1,
    }
  },
  {
    "$lookup": {
      "from": "users",
      "localField": "_id",
      "foreignField": "_id",
      "as": "merged_info"
    }
  },
  {
    "$unwind": {
      "path": "$merged_info",
      "preserveNullAndEmptyArrays": True
    }
  },
  {
    "$replaceRoot": {
      "newRoot": {
        "$mergeObjects": [
          "$merged_info",
          "$$ROOT"
        ]
      }
    }
  },
  {
    "$project": {
      "merged_info": 0,
      "tweet_ids": 0,
      "last_tweet_published_id": 0,
      "user_id": 0
    }
  }
]

In [ ]:
users_labeled_with_original_values = user_collection.aggregate(pipeline)

In [ ]:
users_labeled_with_original_values = list(users_labeled_with_original_values)

Ensuite, on visualise sous la forme d'un histogramme multiple la distribution de chaque attribut en fonction de leur label.

L'histogramme multiple est créé en superposant deux histogrammes, l'un pour les données ayant le label 0 et l'autre pour les données ayant le label 1. Chaque attribut est représenté sur l'axe des x, et le nombre d'occurrences est représenté sur l'axe des y. Les données correspondantes à chaque label sont colorées différemment pour une meilleure distinction visuelle.

En visualisant ces histogrammes, nous pouvons observer la répartition des valeurs pour chaque attribut en fonction de leur label. Cela nous permet de déceler des différences potentielles dans la distribution des données entre les deux catégories. Par exemple, nous pourrions identifier des attributs qui ont des valeurs plus élevées ou plus basses pour un label particulier, ce qui pourrait être utile pour comprendre les caractéristiques distinctives de chaque catégorie.

In [ ]:
label_0_data = [doc for doc in users_labeled_with_original_values if doc['label'] == 0]
label_1_data = [doc for doc in users_labeled_with_original_values if doc['label'] == 1]

# Liste des attributs à visualiser (à adapter en fonction de vos besoins)
attributes = list(users_labeled_with_original_values[0].keys())
attributes.remove('_id')
attributes.remove('label')

# Couleurs pour les deux catégories de labels
colors = ['blue', 'red']

# Création de la grille de graphiques avec deux colonnes
fig, axes = plt.subplots(nrows=len(attributes)//2, ncols=2, figsize=(20, 50))

# Création de l'histogramme multiple
for i, attr in enumerate(attributes):
    ax = axes[i//2][i%2]

    ax.hist([doc[attr] for doc in label_0_data], bins=10, alpha=0.5, color=colors[0], label='Normal')
    ax.hist([doc[attr] for doc in label_1_data], bins=10, alpha=0.5, color=colors[1], label='Atypique')
    ax.set_xlabel(attr)
    ax.set_ylabel('Count')
    ax.set_yscale('log')
    ax.legend()
    
plt.tight_layout()
plt.show()

## ACP

In [ ]:
pca = PCA()
pca.fit(X)

print(pca.explained_variance_)
print(pca.explained_variance_ratio_)

In [ ]:
eig = pd.DataFrame(
    {
        "Dimension" : ["CP" + str(x + 1) for x in range(X.shape[1])], 
        "Variance expliquée" : pca.explained_variance_,
        "cum. var. expliquée" : np.cumsum(pca.explained_variance_),
        "% variance expliquée" : np.round(pca.explained_variance_ratio_ * 100),
        "% cum. var. expliquée" : np.round(np.cumsum(pca.explained_variance_ratio_) * 100)
    }
)
eig

In [ ]:
eig.plot.bar(x = "Dimension", y = "% cum. var. expliquée") # permet un diagramme en barres
plt.axhline(y = 80, linewidth = .5, color = "dimgray", linestyle = "--")
plt.axhline(y = 90, linewidth = .5, color = "dimgray", linestyle = "--")
plt.show()

In [ ]:
n_components = 10  # Remplacez par le nombre de composantes principales souhaitées
X_pca = pca.transform(X)[:, :n_components]
print(X_pca)

## Séparation des données labélisées en Apprentisage , Test et Validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, Y, test_size=0.3)

## Classification avec SVM

### Déterminaison de la meilleure combinaison d'hyperparamètres

Définition d'une fonction pour afficher une matrice de confusion

In [ ]:
def display_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    
    # Création de la figure
    fig, ax = plt.subplots()

    # Création de la heatmap
    heatmap = ax.imshow(cm, cmap='Blues')

    # Ajout des valeurs dans les cellules de la heatmap
    for i in range(len(cm)):
        for j in range(len(cm[i])):
            ax.text(j, i, cm[i][j], ha='center', va='center', color='black')

    # Définition des étiquettes des axes
    classes = ['Normal', 'Atypique']
    ax.set_xticks(range(len(classes)))
    ax.set_yticks(range(len(classes)))
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)

    # Ajout d'une barre de couleur
    cbar = ax.figure.colorbar(heatmap, ax=ax)

    # Ajout des titres
    ax.set_xlabel('Prédictions')
    ax.set_ylabel('Vraies étiquettes')
    ax.set_title('Matrice de confusion')

    # Affichage de la figure
    plt.show()


Définition des hyperparamètres à essayer

In [ ]:
parameters = {
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'C': [1e-2, 1e-1, 1, 1e1],
    'gamma': ['scale', 'auto']
}

Dans notre cas où la classe négative représente environ 84% des échantillons, utiliser le rappel seul pourrait être trompeur. Le rappel mesure la capacité d'un modèle à identifier correctement les échantillons positifs parmi tous les échantillons positifs réels. 

En utilisant ``balanced_accuracy``, on donne une importance égale aux performances des deux classes. Cela permet de s'assurer que notre modèle ne se concentre pas uniquement sur la classe majoritaire (négatif), mais qu'il est également capable de prédire correctement la classe minoritaire (postive). 

Instanciations

In [ ]:
svmc = SVC()
grille = GridSearchCV(estimator=svmc, param_grid=parameters, scoring='balanced_accuracy', cv=2, verbose=3)

Exécuter la recherche de grille pour trouver la meilleure configuration de modèle en ajustant les modèles sur les données d'apprentissage et en évaluant leur performance à l'aide de la validation croisée

In [ ]:
resultats = grille.fit(X_train, y_train)

Affichage du meilleur modèle

In [ ]:
print('Le meilleur modèle :', resultats.best_params_)

In [ ]:
resultats.cv_results_ 

In [ ]:
svm = resultats.best_estimator_
y_true = y_test
y_pred = svm.predict(X_test)

In [ ]:
display_confusion_matrix(y_true, y_pred)

In [ ]:
accuracy_score(y_true,y_pred)

In [ ]:
print(classification_report(y_true,y_pred))

Affichage des moyennes des scores de la cross validation sous la forme d'une heatmap.

In [ ]:
# Résultats du GridSearchCV
results = grille.cv_results_

# Paramètres à afficher sur l'axe des x et des y
C_values = parameters['C']
gamma_values = parameters['gamma']
kernels = parameters['kernel']

# Nombre de kernels et de paramètres sur l'axe des x (C)
n_kernels = len(kernels)
n_C_values = len(C_values)

# Création de la figure
fig, axs = plt.subplots(nrows=n_kernels // 2, ncols=2, figsize=(10, 8))

tuples_list = [(score, params) for score, params in zip(results['mean_test_score'], results['params'])]

# Création des heatmaps pour chaque kernel
for i, kernel in enumerate(kernels):
    # Calcul de l'indice de la facette (subplot)
    row = i // 2
    col = i % 2
    
    # Création d'un dictionnaire pour stocker les scores par combinaison de gamma et C pour un kernel donné
    score_dict = {(params['gamma'], params['C']): score for score, params in tuples_list if params['kernel'] == kernel}
    # Calcul des scores moyens pour chaque combinaison de gamma et C
    scores = [[score_dict.get((gamma, c), None) for c in C_values] for gamma in gamma_values]
    # Création de la heatmap
    ax = axs[row, col]
    heatmap = ax.imshow(scores, cmap='Blues', origin='lower')

    # Ajout des valeurs dans les cellules de la heatmap
    for j, gamma in enumerate(gamma_values):
        for k, c in enumerate(C_values):
            score = scores[j][k]
            if score is not None:
                ax.text(k, j, f'{score:.3f}', ha='center', va='center', color='black')

    # Définition des étiquettes des axes + titre
    ax.set_xticks(range(len(C_values)))
    ax.set_yticks(range(len(gamma_values)))
    ax.set_xticklabels(C_values)
    ax.set_yticklabels(gamma_values)
    ax.set_xlabel('C')
    ax.set_ylabel('gamma')
    ax.set_title(kernel)

# Ajustement des espacements entre les sous-graphiques
plt.tight_layout()

# Ajout de la barre de couleur commune
cbar = fig.colorbar(heatmap, ax=axs.ravel().tolist(), shrink=0.6)
cbar.set_label('Score moyen')

# Affichage de la figure
plt.show()


On regarde si on aurait obtenu de meilleurs résultats/un autre meilleur kernel avec d'autres fonctions de scoring pour le GridSearch

In [ ]:
def test_scoring_value(scoring):
    print('>>> Scoring => ', scoring)
    # determination du meilleur kernel pour la fonction de scoring
    svmc = SVC()
    grille = GridSearchCV(estimator=svmc, param_grid=parameters, scoring=scoring, cv=2)
    resultats = grille.fit(X_train, y_train)
    print('Le meilleur modèle :', resultats.best_params_)
    
    # confusion matrix & accuracy
    svm = resultats.best_estimator_
    y_true = y_test
    y_pred = svm.predict(X_test)
    print(confusion_matrix(y_true, y_pred))
    print(accuracy_score(y_true,y_pred))

In [ ]:
scorings = ['accuracy', 'balanced_accuracy', 'top_k_accuracy', 'average_precision', 'neg_brier_score', 'f1', 'neg_log_loss', 'precision', 'recall', 'jaccard', 'roc_auc']
for s in scorings:
    test_scoring_value(s)

###  Vérification des performances du modèle

Récupération des hyperparamètres optimaux

In [ ]:
try:
    C_opti = resultats.best_params_['C']
    kernel_opti = resultats.best_params_['kernel']
    gamma_opti = resultats.best_params_['gamma']
except:
    C_opti = 10
    kernel_opti = 'rbf'
    gamma_opti = 'auto'

In [ ]:
svm=SVC(C=C_opti,kernel=kernel_opti, gamma=gamma_opti)
svm.fit(X_train, y_train)
y_pred=svm.predict(X_test)
y_true = y_test
erreur=1-accuracy_score(y_test,y_pred)
print(erreur)

display_confusion_matrix(y_true, y_pred)

In [ ]:
accuracy_score(y_true,y_pred)

In [ ]:
print(classification_report(y_true,y_pred))

## Prédictions sur les utilisateurs non labellisés

Nous cherchons à attribuer des labels aux utilisateurs qui n'ont pas encore été labellisés. 

Dans un premier temps, nous allons récupérer les identifiants des utilisateurs déjà labellisés afin de filtrer uniquement ceux qui n'ont pas encore été étiquetés.

In [ ]:
id_labeled_users = users['_id'].tolist()
len(id_labeled_users)

Après avoir obtenu la liste des identifiants des utilisateurs déjà labellisés, nous procédons ensuite à la récupération des utilisateurs qui ne figurent pas dans cette liste. 

En d'autres termes, nous filtrons les utilisateurs en excluant ceux qui ont un identifiant présent dans la liste précédemment obtenue.

In [ ]:
users_scaled_collection = db["users_scaled"]
filtre = {'_id': {'$nin': id_labeled_users}}
unlabeled_users = users_scaled_collection.find(filtre)

Nous allons créer une nouvelle base de données destinée à stocker les utilisateurs nouvellement labélisés. Cette base de données servira à stocker les informations des utilisateurs, y compris les labels qui leur ont été attribués.

In [ ]:
user_predicted_collection = db["users_predicted"]

On supprime toute la collection pour supprimer par la même occasion les données qu'elle contient.

In [ ]:
user_predicted_collection.drop()

Nous allons définir une fonction qui permet de prédire les labels pour un ensemble d'utilisateurs, puis de les insérer dans la nouvelle collection avec leur label. Cette fonction prend en paramètres les utilisateurs non labélisés.

In [ ]:
def user_labeling(unlabeled_users):
    df_batch = pd.DataFrame(unlabeled_users)
    X_batch = df_batch.loc[:, X.columns]
    
    X_pca = pca.transform(X_batch)[:, :n_components]
    labels_pred = svm.predict(X_pca)
    
    for i, user in enumerate(unlabeled_users):
        user['label'] = labels_pred[i].item()

    user_predicted_collection.insert_many(unlabeled_users)

Nous itérons à travers les utilisateurs non labellisés, en leur attribuant des labels à l'aide du SVM préalablement entraîné, puis nous insérons les utilisateurs labellisés par lots dans cette nouvelle collection. Ces opérations sont effectuées à l'aide de la fonction définit dans la cellule précédente.

In [ ]:
batch_size = 100_000
nb_batch = 0
users_batch = []

for i,user in enumerate(unlabeled_users):
    users_batch.append(user)
    if len(users_batch) >= batch_size:
        user_labeling(users_batch)
        users_batch = []
        nb_batch += 1
        print(f'processed {nb_batch*batch_size} users')
        
if len(users_batch) > 0:
    user_labeling(users_batch)

print('END')

## Représentation graphique (temporaire)

### Test de toutes les combinaisons : 3 axes

L'ACP ne nous a pas fournit trois axes principaux pertinents pour visualiser les données, nous avons alors décidé d'explorer toutes les combinaisons de trois axes afin de trouver la meilleure représentation graphique. Cette approche itérative peut révéler des structures et des relations complexes qui n'auraient pas été détectées autrement, bien que cela puisse demander plus de temps et de ressources.

Récupération des données prédites

In [ ]:
user_predicted_collection = db["users_predicted"]
users_predicted = list(user_predicted_collection.find({}))
users_predicted = pd.DataFrame(users_predicted)

In [ ]:
Y_prediction = users_predicted.label
X_prediction = users_predicted.loc[:, X.columns]

Définition des données à visualiser

In [ ]:
X_visu = pd.concat([X, X_prediction])
Y_visu = pd.concat([Y, Y_prediction])

On récupère les attributs qui seront sur les axes

In [ ]:
attributs = X_visu.columns.to_list()
attributs

On détermine l'ensemble des couples de 3 axes possibles

In [ ]:
couples_axes = list(combinations(attributs, 3))

Affichage du graphique correspondant pour chaque combinaison d'axes

In [ ]:
# Création des graphiques 3D pour chaque combinaison d'axes
for couple in couples_axes:
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Extraire les colonnes correspondant au couple d'axes
    x = X_visu[couple[0]]
    y = X_visu[couple[1]]
    z = X_visu[couple[2]]

    # Déterminer les couleurs en fonction de la liste Y
    colors = ['blue' if label == 0 else 'red' for label in Y]

    # Créer le graphique 3D
    ax.scatter(x, y, z, c=colors)

    # Étiquettes des axes
    ax.set_xlabel(couple[0])
    ax.set_ylabel(couple[1])
    ax.set_zlabel(couple[2])

    # Titre du graphique
    title = f"Graphique 3D ({couple[0]}, {couple[1]}, {couple[2]})"
    ax.set_title(title)
    
    # Légende
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label='Normaux', markerfacecolor='blue', markersize=8),
                       plt.Line2D([0], [0], marker='o', color='w', label='Atypiques', markerfacecolor='red', markersize=8)]
    ax.legend(handles=legend_elements)

    # Afficher le graphique
    plt.show()

### Anciennement

In [ ]:

scatter = plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test,edgecolors='k', cmap=plt.cm.coolwarm)
legend = plt.legend(*scatter.legend_elements(), title='Label')
# Afficher le graphique 2D
plt.xlabel('x1')
plt.ylabel('x2')
plt.title("SVM 2D Données test")
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Tracer les points en 3D avec une couleur basée sur la dimension supplémentaire
sc = ax.scatter(X_test[:, 0], X_test[:, 1],X_test[:, 2], c=y_train, cmap=plt.cm.coolwarm,edgecolors='k')
legend = ax.legend(*sc.legend_elements(), title='Label')
ax.add_artist(legend)

# Ajouter des labels aux axes
ax.set_xlabel('Dim 1')
ax.set_ylabel('Dim 2')
ax.set_zlabel('Dim 3')
ax.title("SVM 3D Données test")

# Afficher le graphique 3D
plt.show()